In [20]:
import pandas as pd
from currency_converter import CurrencyConverter

from constants import DATAFRAMES

# Data Cleaning

The goal is to fix or handle data quality issues identified in the validation stage.

NOTE: From data cleaning stage onwards, I will work with `clients.csv`, `trader.csv` and `trans.csv`. The `a2p_ref.csv` and `partner_codes.csv` are more suitable for partner attrition analysis which is not part of the assignment.

In [21]:
display(f'Initial record counts:')
for name, df in DATAFRAMES.items():
    display(f'{name}: {len(df)}')

'Initial record counts:'

'df_trans: 7855'

'df_trader: 7804'

'df_clients: 5327'

'df_partner_codes: 1592'

'df_a2p_ref: 7733'

## 1. Filter Invalid Transaction Amounts

In [22]:
invalid_amounts = DATAFRAMES['df_trans'][DATAFRAMES['df_trans'].amount <= 0]
display(f"Found {len(invalid_amounts)} transactions with amount <= 0")
display(invalid_amounts.head())

'Found 1 transactions with amount <= 0'

,login,created_at,transaction_id,amount,currency
1523,6085899,2025-01-31 11:25:00,17046055,-10000.0,JPY


In [23]:
df_trans_clean = DATAFRAMES['df_trans'][DATAFRAMES['df_trans'].amount > 0].copy()
display(f"Removed {len(DATAFRAMES['df_trans']) - len(df_trans_clean)} transactions with invalid amounts")
display(f"Remaining transactions: {len(df_trans_clean)}")

'Removed 1 transactions with invalid amounts'

'Remaining transactions: 7854'

## 2. Filter Invalid Registration Types

In [24]:
display("Registration types found:")
display(DATAFRAMES['df_clients'].type.value_counts())

'Registration types found:'

type
Full     4530
Demo      431
Light     364
0           2
Name: count, dtype: int64

In [25]:
valid_types = ['Full', 'Light', 'Demo']
invalid_types = DATAFRAMES['df_clients'][~DATAFRAMES['df_clients'].type.isin(valid_types)]
display(f"Found {len(invalid_types)} clients with invalid registration types")

'Found 2 clients with invalid registration types'

In [26]:
df_clients_clean = DATAFRAMES['df_clients'][DATAFRAMES['df_clients'].type.isin(valid_types)].copy()
display(f"Removed {len(DATAFRAMES['df_clients']) - len(df_clients_clean)} clients with invalid registration types")
display(f"Remaining clients: {len(df_clients_clean)}")

'Removed 2 clients with invalid registration types'

'Remaining clients: 5325'

## 3. Remove Orphaned Records - Referential Integrity

### 3.1 Remove traders with non-existent clients

In [27]:
orphaned_traders = DATAFRAMES['df_trader'][~DATAFRAMES['df_trader'].client.isin(df_clients_clean.client)]
display(f"Found {len(orphaned_traders)} trader records with non-existent clients")
display(f"Percentage: {len(orphaned_traders) / len(DATAFRAMES['df_trader']) * 100:.2f}%")

'Found 40 trader records with non-existent clients'

'Percentage: 0.51%'

In [28]:
df_trader_clean = DATAFRAMES['df_trader'][DATAFRAMES['df_trader'].client.isin(df_clients_clean.client)].copy()
display(f"Removed {len(DATAFRAMES['df_trader']) - len(df_trader_clean)} orphaned trader records")
display(f"Remaining traders: {len(df_trader_clean)}")

'Removed 40 orphaned trader records'

'Remaining traders: 7764'

### 3.2 Remove transactions with non-existent logins

In [29]:
orphaned_trans = df_trans_clean[~df_trans_clean.login.isin(df_trader_clean.login)]
display(f"Found {len(orphaned_trans)} transactions with non-existent logins")
display(f"Percentage: {len(orphaned_trans) / len(df_trans_clean) * 100:.2f}%")

'Found 91 transactions with non-existent logins'

'Percentage: 1.16%'

In [30]:
df_trans_clean = df_trans_clean[df_trans_clean.login.isin(df_trader_clean.login)].copy()
display(f"Removed {len(orphaned_trans)} orphaned transaction records")
display(f"Remaining transactions: {len(df_trans_clean)}")

'Removed 91 orphaned transaction records'

'Remaining transactions: 7763'

### 3.3 Verify first_deposit_id references

In [31]:
invalid_first_deposits = df_trader_clean[df_trader_clean.first_deposit_id.notna() & ~df_trader_clean.first_deposit_id.isin(df_trans_clean.transaction_id)]
display(f"Found {len(invalid_first_deposits)} traders with invalid first_deposit_id references")


'Found 1 traders with invalid first_deposit_id references'

In [32]:
if len(invalid_first_deposits) > 0:
    display("Setting invalid first_deposit_id to null")
    df_trader_clean.loc[
        (invalid_first_deposits.login.values == df_trader_clean.login.values) &
        (invalid_first_deposits.client.values == df_trader_clean.client.values) &
        (invalid_first_deposits.account_id.values == df_trader_clean.account_id.values), 'first_deposit_id'] = None
    display(f"Set {len(invalid_first_deposits)} first_deposit_id values to null")

'Setting invalid first_deposit_id to null'

'Set 1 first_deposit_id values to null'

## 4. Data Type Conversions

In [33]:
id_columns = ['account_id', 'first_deposit_id']
for col in id_columns:
    df_trader_clean[col] = df_trader_clean[col].astype('Int64')

In [34]:
display(df_trader_clean.info())
display(df_trader_clean.head())

<class 'pandas.core.frame.DataFrame'>
Index: 7764 entries, 0 to 7803
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   login             7764 non-null   int64
 1   client            7764 non-null   int64
 2   account_id        7605 non-null   Int64
 3   first_deposit_id  7702 non-null   Int64
dtypes: Int64(2), int64(2)
memory usage: 318.4 KB


None

,login,client,account_id,first_deposit_id
0,3639642,25831,570086139,37032272
1,3648174,75264,702559253,37112579
2,3650697,89278,717548133,35428843
3,3651651,52437,733441170,35186270
4,3658908,32725,820931437,36992110


In [35]:
df_trans_clean['amount'] = df_trans_clean['amount'].astype('Float64')

In [36]:
display(df_trans_clean.info())
display(df_trans_clean.head())

<class 'pandas.core.frame.DataFrame'>
Index: 7763 entries, 0 to 7854
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   login           7763 non-null   int64         
 1   created_at      7763 non-null   datetime64[ns]
 2   transaction_id  7763 non-null   int64         
 3   amount          7763 non-null   Float64       
 4   currency        7763 non-null   object        
dtypes: Float64(1), datetime64[ns](1), int64(2), object(1)
memory usage: 371.5+ KB


None

,login,created_at,transaction_id,amount,currency
0,27061833,2025-02-27 16:24:00,13203755,10000.0,JPY
1,6087675,2025-02-28 16:04:00,17434895,100000.0,JPY
2,5471727,2025-02-28 16:16:00,14728768,30000.0,JPY
3,6634899,2025-02-28 16:29:00,6104013,30000.0,JPY
4,6629259,2025-02-27 21:31:00,6095829,50000.0,JPY


## 5. Handle Missing Values

In [37]:
display("Missing values in df_trans_clean:")
display(df_trans_clean.isnull().sum())

display("\nMissing values in df_trader_clean:")
display(df_trader_clean.isnull().sum())

display("\nMissing values in df_clients_clean:")
display(df_clients_clean.isnull().sum())

'Missing values in df_trans_clean:'

login             0
created_at        0
transaction_id    0
amount            0
currency          0
dtype: int64

'\nMissing values in df_trader_clean:'

login                 0
client                0
account_id          159
first_deposit_id     62
dtype: int64

'\nMissing values in df_clients_clean:'

client         0
client_id      0
_created_on    0
type           0
dtype: int64

NOTE: account_id and first_deposit_id can be null - this is acceptable. 
 - the trader registered but never opened an account
 - the trader has an account but hasn't made a deposit yet

## 6. Handle currency conversion

In [38]:
c = CurrencyConverter(fallback_on_missing_rate=True)
df_trans_clean['amount_USD'] = df_trans_clean.apply(
    lambda x: c.convert(x['amount'], x['currency'], 'USD', date=x['created_at'].date()), axis=1
)

In [40]:
display(df_trans_clean.info())
display(df_trans_clean.head())

<class 'pandas.core.frame.DataFrame'>
Index: 7763 entries, 0 to 7854
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   login           7763 non-null   int64         
 1   created_at      7763 non-null   datetime64[ns]
 2   transaction_id  7763 non-null   int64         
 3   amount          7763 non-null   Float64       
 4   currency        7763 non-null   object        
 5   amount_USD      7763 non-null   float64       
dtypes: Float64(1), datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 432.1+ KB


None

,login,created_at,transaction_id,amount,currency,amount_USD
0,27061833,2025-02-27 16:24:00,13203755,10000.0,JPY,66.847445
1,6087675,2025-02-28 16:04:00,17434895,100000.0,JPY,663.290010
2,5471727,2025-02-28 16:16:00,14728768,30000.0,JPY,198.987003
3,6634899,2025-02-28 16:29:00,6104013,30000.0,JPY,198.987003
4,6629259,2025-02-27 21:31:00,6095829,50000.0,JPY,334.237223
